# 读取数据

In [11]:
import pandas as pd

file_path = '/kaggle/input/e-commerce-user-behavior-dataset/ecommerce_user_behavior_dataset.csv'

In [12]:
# 如果需要保留中英文列名对应关系：
# 先读取第一行(英文列名)
with open(file_path, 'r', encoding='gbk') as f:
    en_columns = f.readline().strip().split(',')
    cn_columns = f.readline().strip().split(',')

# 创建列名映射字典
col_mapping = dict(zip(en_columns, cn_columns))
print("\n中英文列名对应关系：")
for en, cn in col_mapping.items():
    print(f"{en}: {cn}")

# 读取实际数据(跳过前两行)
df = pd.read_csv(file_path, 
                skiprows=2, 
                header=None, 
                names=en_columns,
                encoding='gbk')


中英文列名对应关系：
User_ID: 每个用户的唯一标识符，便于追踪和分析。
Age: 用户的年龄，提供对人口统计偏好的洞察。
Gender: 用户的性别，使能性别特定的推荐和定位。
Location: 用户所在地区：郊区、农村、城市，影响偏好和购物习惯。
Income: 用户的收入水平，表明购买力和支付能力。
Interests: 用户的兴趣，如运动、时尚、技术等，指导内容和产品推荐。
Last_Login_Days_Ago: 用户上次登录以来的天数，反映参与频率。
Purchase_Frequency: 用户进行购买的频率，表明购物习惯和忠诚度。
Average_Order_Value: 用户下单的平均价值，对定价和促销策略至关重要。
Total_Spending: 用户消费的总金额，表明终身价值和购买行为。
Product_Category_Preference: 用户偏好的特定产品类别。
Time_Spent_on_Site_Minutes: 用户在电子商务平台上花费的时间，表明参与程度。
Pages_Viewed: 用户在访问期间浏览的页面数量，反映浏览活动和兴趣。
Newsletter_Subscription: 用户是否订阅了营销活动通知。


# RFM模型建立

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# 数据预处理
# 确保数值列正确转换
df['Last_Login_Days_Ago'] = pd.to_numeric(df['Last_Login_Days_Ago'], errors='coerce')
df['Purchase_Frequency'] = pd.to_numeric(df['Purchase_Frequency'], errors='coerce')
df['Total_Spending'] = pd.to_numeric(df['Total_Spending'], errors='coerce')

#计算RFM指标
rfm = df.groupby('User_ID').agg({
    'Last_Login_Days_Ago': 'min',  # 使用min()因为已经是天数差
    'Purchase_Frequency': 'sum',
    'Total_Spending': 'sum'
}).rename(columns={
    'Last_Login_Days_Ago': 'Recency',
    'Purchase_Frequency': 'Frequency',
    'Total_Spending': 'Monetary'
})

rfm = rfm.fillna({
    'Recency': df['Last_Login_Days_Ago'].max()+30,  # 从未登录的用户设为"比最久还久30天"
    'Frequency': 0,  # 无消费记录的用户频次为0
    'Monetary': 0    # 无消费记录的用户金额为0
})

# 数据转换
# Recency越小表示越近，所以取负值（确保已经是数值类型）
rfm['Recency'] = -rfm['Recency'].astype(float)


# 标准化处理
scaler = StandardScaler()
rfm_scaled = pd.DataFrame(scaler.fit_transform(rfm), 
             columns=rfm.columns, 
             index=rfm.index)

# K-means模型建立

In [5]:
# K-Means聚类
kmeans = KMeans(n_clusters=4, random_state=42)
rfm['Cluster'] = kmeans.fit_predict(rfm_scaled)

# 定义用户类型标签
cluster_labels = {
    0: '高价值用户',
    1: '潜力用户',
    2: '一般保持用户',
    3: '流失风险用户'
}
rfm['User_Type'] = rfm['Cluster'].map(cluster_labels)

# 合并回原始数据
user_profiles = df.merge(rfm[['Cluster', 'User_Type']], 
                       left_on='User_ID', 
                       right_index=True,
                       how='left')

# 分析结果
print("用户类型分布:")
print(user_profiles['User_Type'].value_counts())

# 各群体RFM特征
print("\n各群体RFM均值:")
print(rfm.groupby('User_Type').mean())

用户类型分布:
User_Type
潜力用户      269
高价值用户     260
一般保持用户    252
流失风险用户    219
Name: count, dtype: int64

各群体RFM均值:
             Recency  Frequency     Monetary  Cluster
User_Type                                            
一般保持用户    -22.273810   5.309524  1257.250000      2.0
流失风险用户    -18.479452   1.662100  3605.602740      3.0
潜力用户       -6.680297   4.048327  1676.171004      1.0
高价值用户     -15.880769   7.076923  3829.280769      0.0


/usr/local/lib/python3.11/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


# 可视化

## 散点图

In [10]:
import plotly.express as px

# 交互式3D散点图
fig = px.scatter_3d(rfm.reset_index(),
                   x='Recency',
                   y='Frequency',
                   z='Monetary',
                   color='User_Type',
                   hover_name='User_ID',
                   opacity=0.7,
                   title='RFM用户分群三维可视化')

fig.update_layout(scene=dict(
    xaxis_title='近度(负值)',
    yaxis_title='频度',
    zaxis_title='消费金额'
))
fig.show()

## 雷达图

In [9]:
import numpy as np
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler

# 准备雷达图数据
categories = ['Recency','Frequency','Monetary']
cluster_means = rfm.groupby('User_Type')[categories].mean().reset_index()

# Min-Max标准化
scaler = MinMaxScaler()
cluster_means[categories] = scaler.fit_transform(cluster_means[categories])

# 绘制雷达图
fig = go.Figure()

for i in range(len(cluster_means)):
    fig.add_trace(go.Scatterpolar(
        r=cluster_means.iloc[i][categories].values.tolist() + [cluster_means.iloc[i][categories].values[0]],
        theta=categories + [categories[0]],
        fill='toself',
        name=cluster_means.iloc[i]['User_Type'],
        hoverinfo='name+r+theta',
        line=dict(width=2),
        opacity=0.8
    ))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 1],
            tickvals=[0, 0.2, 0.4, 0.6, 0.8, 1],
            ticktext=['0%', '20%', '40%', '60%', '80%', '100%']
        ),
        angularaxis=dict(
            rotation=90  # 调整角度标签的起始位置
        )
    ),
    title=dict(
        text='各用户群体RFM特征雷达图',
        x=0.5,
        xanchor='center',
        font=dict(size=16)
    ),
    showlegend=True,
    legend=dict(
        orientation="v",  # 垂直排列
        yanchor="middle",  # 锚点居中
        y=0.5,            # Y位置居中
        x=1.2,            # 向右偏移
        xanchor="left",   # 以左侧为锚点
        bgcolor='rgba(255,255,255,0.5)',
        bordercolor='#CCC',
        borderwidth=1
    ),
    margin=dict(l=100, r=150),  # 为右侧图例留出空间
    width=700,  # 增加画布宽度
    height=500
)

# 添加注释说明标准化方法
fig.add_annotation(
    x=0.5,
    y=-0.15,
    text="<i>数据经过Min-Max标准化处理（0-1范围）</i>",
    showarrow=False,
    xref="paper",
    yref="paper",
    font=dict(size=10, color='gray')
)

fig.show()